In [ ]:
import warnings, gc
import numpy as np 
import pandas as pd
import matplotlib.colors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff
from tqdm.auto import tqdm
import copy
import math
import random
import matplotlib.pyplot as plt

First we load the array computed in the other notebook. It's a NxM array containing a simulation of daily spreadys for M days of N different portfolios

In [ ]:
arr=np.load('output/daily_spreads.npy')
arr.shape

The array has 679 rows, one for each portfolio and 205 column, one for each day. Each element [i,j] is the daily spread of the portfolio i in the day j

In [ ]:
std=np.std(arr,axis=0)

I should modyfy those functions so that they return the k-th nearest values

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
def find_k_nearest(array,value,k):
    '''this function returns the k nearest values to a targed value of a given array'''
    aux=np.abs(array - value)
    indices=np.argsort(aux)
    idx=indices[:k]
    return array[idx]

In [ ]:
def find_nearest_sorted(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

## Some plots

for each portfolio i from 0 to n make a scatter plot of points with coordinates (day, daily_spread(portfolio(i),day))

In [ ]:
portfs=1000
fig=plt.figure(figsize=(14,8))
for i in range(len(arr[:portfs,0])): #for each portfolio i from 0 to n
    plt.scatter(range(len(arr[0])),arr[i],alpha=0.3,s=1) #make a scatter plot of points with coordinates (day, daily_spread(portfolio(i),day))
plt.plot(std)
plt.xlabel('Day')
plt.ylabel('Daily Spread')
plt.title('Daily spread for portfolios')
plt.show()

In [ ]:
zip()

In [ ]:

fig =px.scatter(x=range(len(arr[0])),y=arr[0])
#fig.add_scatter(x=range(len(arr[0])),y=arr[1])
fig.show()

In [ ]:
fig=plt.figure(figsize=(14,8))
for i in range(len(arr[:,0])): #for each portfolio i from 0 to n
    plt.plot(arr[i],alpha=0.3,linewidth=0.1) #make a scatter plot of points with coordinates (day, daily_spread(portfolio(i),day))
plt.plot(std)
plt.xlabel('Day')
plt.ylabel('Daily Spread')
plt.title('Daily spread for portfolios')
plt.show()

In [ ]:
arr_sorted=np.sort(arr,axis=0)
len(arr_sorted[0])

In [ ]:
find_nearest(arr_sorted[:,0],4)

In [ ]:
np.apply_along_axis(find_nearest,0,arr,4)

In [ ]:
np.apply_along_axis(find_nearest_sorted,0,arr_sorted,4)

In [ ]:
def k_n_array(arr,value,k,axis=0):
    '''returns the k-th nearest values to a certain value along an axis of a given array'''
    def findk(array,value,k):
        aux=np.abs(array - value)
        indices=np.argsort(aux)
        idx=indices[:k]
        return array[idx]
    return np.apply_along_axis(findk,axis,arr,value,k)

In [ ]:
k=10
v=4
np.apply_along_axis(find_k_nearest,0,arr,v,k)

In [ ]:
find_nearest_sorted(arr_sorted[:,0],4)

In [ ]:
d_sp=np.arange(-3,3,step=0.01)

Now, for each day, and for each value of daily spread, I compute the 5 closest values of daily sperad to it achiaved by my portfolios

In [ ]:
vals=[]
for d in tqdm(d_sp):
    vals.append(k_n_array(arr_sorted,d,30))

In [ ]:
def sharpe_dspr(array,knn=30,step=0.01):
    '''returns the sharpe ratio obtained trying to keep
     a constant daily spread for the time period considered
    Input: 
    
    array: an array of shape n_portfolios x m_days
    knn: the number of closest daily spread to average
    '''
    d_sp=np.arange(-3,3,step=step)
    vals=[]
    for d in tqdm(d_sp):
        vals.append(k_n_array(array,d,knn))
    vals=np.array(vals)
    aa=vals.mean(axis=1)
    sharpe=aa.mean(1)/aa.std(1)
    return sharpe

In [ ]:
vals=np.array(vals)
vals.shape

this is an array of dimension (n_steps x n_neighbours x days)

In [ ]:
step=0.005
sharpe=sharpe_dspr(arr,30,step)
d_sp=np.arange(-3,3,step=step)

Let's recreate the plot in https://www.youtube.com/watch?v=51tM3ktXqPY to find the optimal value of daily spread

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(d_sp,sharpe)
step=0.01
plt.plot(np.arange(-3,3,step=step),sharpe_dspr(arr,30,step))
plt.plot(np.arange(-3,3,step=step),sharpe_dspr(arr,10,step))
plt.grid()
plt.xlabel('Daily spread')
plt.ylabel('Sharpe Ratio (arbitrary normalization)')
plt.show()